In [1]:
import numpy as np ; import pandas as pd 
import matplotlib.pyplot as plt 
from seedtools  import load_seed
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler,FunctionTransformer,PowerTransformer,KBinsDiscretizer
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
def score_data(x_train,y_train,x_test,y_test):
    model =  LogisticRegression()
    model.fit(x_train,y_train)
    score=  model.score(x_test,y_test)
    del model 
    return score

**NOTE  : this is not proper machine learning this notebook just have how to** <br />
**apply this in programming but focus on accuracy is not maintained**

### CONTENTS
-  ColumN Transformer
- PipeLine 
- Function Transformer
- Power Transformer
- Binning and Discretization

## Column Tranformer 
##### Column tansformer enables us to make transformation to indivisual columns in a single line 

In [3]:
data = load_seed("adult_mod.csv").data
X  =  data.drop(columns=["Income","Unnamed: 0","Education","Marital Status","Occupation","Relationship","Country"],axis=1)
y=  data.Income
x_train,x_test,y_train,y_test =  train_test_split(X,y,test_size=0.2)
x_train.shape,y_train.shape

C:\Users\RISHABH\AppData\Roaming\Python\Python311\site-packages\seedtools\seed_file.py:190: UserWarning: SEED FILE IS NOT CONFIGURED YET , CONFIGURE IT USING `register` and `register_version`
  warnings.warn("SEED FILE IS NOT CONFIGURED YET , CONFIGURE IT USING `register` and `register_version`")


((26048, 4), (26048,))

In [4]:
column_transformer=  ColumnTransformer(transformers=[
    ('tnf1',OrdinalEncoder(categories=[['Other',' Black',' White']]),['Race']),
    ('tnf2',OneHotEncoder(),['Sex','Workclass']),
    
],remainder="passthrough")

In [5]:

x_train_ct = column_transformer.fit_transform(x_train)
print("x_train_ct shape :",x_train_ct.shape)


x_test_ct =  column_transformer.transform(x_test)
print("x_test_ct shape :",x_test_ct.shape)
x_test_ct[:1]

x_train_ct shape : (26048, 7)
x_test_ct shape : (6513, 7)


array([[ 2.,  0.,  1.,  0.,  1.,  0., 41.]])

In [6]:
score_data(x_train_ct,y_train,x_test_ct,y_test)

0.7425149700598802

**Contains categorical data thus cant fit to model**
<div id ="mypipe"> </div>


## Pipeline

**`Pipeline` requires naming of steps, `make_pipeline` does not.** <br />
(Same applies to `ColumnTransformer` vs `make_column_transformer`)

In [7]:
data  =  load_seed("Social_Network_Ads.csv").data
data =  data.drop(columns=["User ID"])
X = data.drop(columns=["Purchased"])
y = data.Purchased
x_train,x_test,y_train,y_test =  train_test_split(X,y,test_size=0.2)

Shape: [400, 4]
Columns: ['Gender', 'Age', 'EstimatedSalary', 'Purchased']
Description: DATA IS NOT YET PROVIDED


In [8]:
ct1 = ColumnTransformer([
    ("scaling",StandardScaler(),[1,2]),
    ("ohe_gender",OneHotEncoder(handle_unknown="ignore"),[0])
],remainder="passthrough")


ct3 =  RandomForestClassifier()

In [9]:
pipe = Pipeline(steps=[
    ("tp1",ct1),
    ("model",ct3)

])
pipe.fit(x_train,y_train)

,steps,"[('tp1', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scaling', ...), ('ohe_gender', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [10]:
pipe.score(x_test,y_test)

0.8625

## Function Tranformer
**Any mathematical function u give the data will be transformed in same manner**

In [11]:
x = data.drop(columns=["Gender"]).values
y =  data["Purchased"].values
x_train,x_test,y_train,y_test =  train_test_split(x,y,test_size=0.2)


In [12]:
ft =  FunctionTransformer(func=np.log1p)
x_train_ft =  ft.fit_transform(x_train)
x_test_ft =  ft.transform(x_test)

In [13]:
print("Few Values of x_train",x_train[:3][0])
print("Few values of x_train_ft",x_test_ft[:3][0])

Few Values of x_train [   31 76000     0]
Few values of x_train_ft [ 3.66356165 10.81979828  0.        ]


In [14]:
before_trans_score= score_data(x_train,y_train,x_test,y_test)
after_trans_data =  score_data(x_train_ft,y_train,x_test_ft,y_test)
print(before_trans_score,after_trans_data)

1.0 1.0


## Power Transformer 
-  Box-Cox Method
- Yeo-Johnso Method

***BOX COX METHOD*** <br />
=> add `0.0001`  to avoid negative values  <br />
=> this method cant be applied to negative values

In [19]:
x = data.drop(columns=["Gender"])
y =  data["Purchased"].values
x_train,x_test,y_train,y_test =  train_test_split(x.values,y,test_size=0.2)

In [20]:
pt  =  PowerTransformer(method="box-cox")
x_train_pt =  pt.fit_transform(x_train + 0.00001)
x_test_pt =  pt.transform(x_test + 0.00001)
lambdas_df =  pd.DataFrame({"Cols":x.columns,"Box-Cox-lambdas":pt.lambdas_})

In [21]:
before_trans_score= score_data(x_train,y_train,x_test,y_test)
after_trans_data =  score_data(x_train_pt,y_train,x_test_pt,y_test)
print(before_trans_score,after_trans_data)


1.0 1.0


C:\Users\RISHABH\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


***Yeo-Johnson Method***

In [22]:
pt  =  PowerTransformer() # deafult is Yeo-Johnson Method
x_train_pt_1 =  pt.fit_transform(x_train)
x_test_pt_1 =  pt.transform(x_test)
lambdas_df["Yeo-Johnson-lambdas"] =  pt.lambdas_

In [23]:
before_trans_score= score_data(x_train,y_train,x_test,y_test)
after_trans_data =  score_data(x_train_pt_1,y_train,x_test_pt_1,y_test)
print(before_trans_score,after_trans_data)


1.0 1.0


C:\Users\RISHABH\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
lambdas_df.head()

,Cols,Box-Cox-lambdas,Yeo-Johnson-lambdas
0,Age,0.461288,0.448226
1,EstimatedSalary,0.483656,0.483648
2,Purchased,-0.205951,-3.420768


## Discretization and Binarization 

**types of binning** :
- supervised => equal width(uniform) , equal freq (percentile), KMeans Binning(new random cluster wise  )
- unsupervised => Decision Tree Bining


In [25]:
x = data.drop(columns=["Gender"])
y =  data["Purchased"]

In [26]:
kbins_age =  KBinsDiscretizer(n_bins=10,encode="ordinal",strategy="quantile")
kbins_sal =  KBinsDiscretizer(n_bins=10,encode="ordinal",strategy="quantile")

x["Age_bins"] =  kbins_age.fit_transform(x['Age'].values.reshape(-1,1))
x["sal_bins"] =  kbins_sal.fit_transform(x['EstimatedSalary'].values.reshape(-1,1))

C:\Users\RISHABH\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
C:\Users\RISHABH\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


In [27]:
X_after =  x[["Age_bins","sal_bins"]].values
y =  x["Purchased"].values
x_train_after,x_test_after,y_train,y_test =  train_test_split(X_after,y,test_size=0.2)

X_before =  x[["Age_bins","sal_bins"]].values
y =  x["Purchased"].values
x_train_before,x_test_before,y_train,y_test =  train_test_split(X_before,y,test_size=0.2)

In [28]:
before_trans_score= score_data(x_train_before,y_train,x_test_before,y_test)
after_trans_data =  score_data(x_train_after,y_train,x_test_after,y_test)
print(before_trans_score,after_trans_data)

0.7875 0.6375
